## 1 Datenerhebung mittels API & Web Scraping

In [7]:
#import modules
import pandas as pd
import numpy as np
import seaborn as sns
#sns.get_dataset_names()
from pandas_datareader import data
from sklearn.model_selection import train_test_split
import time
import datetime

### 1.1 Yahoo Finance API: Aktienkurs

In [8]:
ticker = 'AAPL'
period1 = int(time.mktime(datetime.datetime(2010, 1, 1, 23, 59).timetuple()))
period2 = int(time.mktime(datetime.datetime(2022, 2, 1, 23, 59).timetuple()))
interval = '1d'
query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'
data = pd.read_csv(query_string)
print(data)
data.to_csv('APPL Prices.csv')

            Date        Open        High         Low       Close   Adj Close  \
0     2010-01-04    7.622500    7.660714    7.585000    7.643214    6.505280   
1     2010-01-05    7.664286    7.699643    7.616071    7.656429    6.516526   
2     2010-01-06    7.656429    7.686786    7.526786    7.534643    6.412873   
3     2010-01-07    7.562500    7.571429    7.466071    7.520714    6.401016   
4     2010-01-08    7.510714    7.571429    7.466429    7.570714    6.443575   
...          ...         ...         ...         ...         ...         ...   
3037  2022-01-26  163.500000  164.389999  157.820007  159.690002  158.526489   
3038  2022-01-27  162.449997  163.839996  158.279999  159.220001  158.059906   
3039  2022-01-28  165.710007  170.350006  162.800003  170.330002  169.088974   
3040  2022-01-31  170.160004  175.000000  169.509995  174.779999  173.506546   
3041  2022-02-01  174.009995  174.839996  172.309998  174.610001  173.337799   

         Volume  
0     493729600  
1  

### 1.2 Web Scraping

In [9]:
# import modules
import os 
import requests 
import pandas as pd 
from bs4 import BeautifulSoup

#get the URL using response variable 
my_url = "https://finance.yahoo.com/quote/AAPL/community?p=AAPL" 
response = requests.get(my_url)

#Catching Exceptions 
print("response.ok : {} , response.status_code : {}".format(response.ok , response.status_code)) 
print("Preview of response.text : ", response.text[:500])


###########################
import requests
from bs4 import BeautifulSoup

# die URL der Webseite, die gescraped werden soll
url = "https://www.cash.ch/news/top-news"

# die HTML-Seite herunterladen
response = requests.get(url)

# eine BeautifulSoup-Instanz erstellen
soup = BeautifulSoup(response.content, "html.parser")

# alle Schlagzeilen auf der Seite auswählen
headlines = soup.find_all("h2", {"class": "media-heading"})

# alle Schlagzeilen ausgeben
for headline in headlines:
    print(headline.text.strip())


response.ok : False , response.status_code : 404
Preview of response.text :  <!DOCTYPE html>
  <html lang="en-us"><head>
  <meta http-equiv="content-type" content="text/html; charset=UTF-8">
      <meta charset="utf-8">
      <title>Yahoo</title>
      <meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui">
      <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
      <style>
  html {
      height: 100%;
  }
  body {
      background: #fafafc url(https://s.yimg.com/nn/img/sad-panda-201402200631.png) 50% 50%;
      background-size: cove


## 2 Datenaufbereitung

### Entfernen NAs und Duplikate, Erstellen neuer Variablen, Anreicherung der Daten

In [10]:
# Data cleaning --> Hier noch mehr Befehle suchen

df = data.drop_duplicates()
df['Date'] = pd.to_datetime(df['Date'])

## 3 DB - Postgres DB initiate -> In Docker

In [11]:
# Libraries
import os
import fnmatch
import tempfile
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

os.environ['MPLCONFIGDIR'] = "/home/jovyan"
import matplotlib.pyplot as plt

# Settings
import warnings
warnings.filterwarnings("ignore")

### DB Connect

In [12]:
conn = psycopg2.connect("host=db dbname=postgres user=admin password=secret")

OperationalError: could not translate host name "db" to address: Unknown host


### 3.1 DB Insert

In [ ]:
engine = create_engine('postgresql://admin:secret@db:5432/postgres')
data.to_sql('appl_prices', engine, if_exists='replace')

### 3.2 SQL Abfrage Select *

In [ ]:
df_sub = pd.read_sql_query('''SELECT
                             *
                             FROM appl_prices''', 
                          con=engine)
df_sub.head()
df_sub.count()

## 4. EDA

In [ ]:
#Daten aus DB lesen und bearbeiten
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_datareader import data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# Exploratory data analysis
print(df.info())
print(df.describe())

# Plotting
sns.set_style('whitegrid')
plt.figure(figsize=(12,6))
plt.title('Apple Stock Price')
plt.xlabel('Year')

plt.ylabel('Price ($)')
sns.lineplot(data=df, x='Date', y='Close')
plt.show()

plt.figure(figsize=(12,6))
plt.title('Daily Change in Apple Stock Price')
plt.xlabel('Year')
plt.ylabel('Change in price ($)')
sns.lineplot(data=df, x='Date', y='Close').set(ylabel='Price ($)', xlabel='Year')
sns.lineplot(data=df, x='Date', y=df['Close'].diff()).set(ylabel='Change in price ($)', xlabel='Year')
plt.legend(labels=['Price', 'Daily Change'])
plt.show()

plt.figure(figsize=(12,6))
plt.title('Apple Stock Price Distribution')
sns.histplot(data=df, x='Close', bins=30)
plt.show()

# Split the data into training and test sets
X = df['Open'].values.reshape(-1, 1)
y = df['Close'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on test data
y_pred = model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print(f'R-squared: {r2:.2f}')
print

### 5. ML Framework

### Z.3 Integration und Visualisierung von geographischen Daten

In [ ]:
import yfinance as yf
import folium
import requests
import webbrowser
import os
from bs4 import BeautifulSoup

# Get the Exchange from Yahoo Finance
ticker = yf.Ticker('AAPL').info
market_place = ticker['exchange']
print('Ticker:', ticker)
print('Ticker: AAPL')
print('Market Place:', market_place)

# Yahoo Finance API URL to get exchange symbols for AAPL stock
yahoo_api_url = 'https://finance.yahoo.com/quote/AAPL'

# Nominatim API URL to get geocoding data for exchange locations
nominatim_api_url = 'https://nominatim.openstreetmap.org/search'

# Get exchange symbols for AAPL stock
response = requests.get(yahoo_api_url)
soup = BeautifulSoup(response.content, 'html.parser')
exchange_symbols = market_place
print(exchange_symbols)

# OpenStreetMap URL to get location data for NMS stock exchange
osm_url = f'https://nominatim.openstreetmap.org/search.php?q={exchange_symbols}+stock+exchange&format=json'

# Get location data for NMS stock exchange
response = requests.get(osm_url)
location_data = response.json()[0]

# Extract latitude and longitude from location data
lat = float(location_data['lat'])
lon = float(location_data['lon'])

# Create a folium map centered on the NMS stock exchange
m = folium.Map(location=[lat, lon], zoom_start=16)

# Add a marker for the NMS stock exchange
folium.Marker(location=[lat, lon], tooltip='NMS stock exchange').add_to(m)

# Display the map
m
m.save('Exchange.html')
url = 'file://' + os.path.abspath('Exchange.html')
webbrowser.open(url)